In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可: 
# Also add the following code, 
# so that every time the environment (kernel) starts, 
# just run the following code: 
import sys 
sys.path.append('/home/aistudio/external-libraries')

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 准备数据和训练模型
- 安装PaddleDetection和其使用依赖
- 解压比赛数据
- 按照yolo框架需要格式处理数据并切分数据集

In [ ]:
! git clone https://gitee.com/paddlepaddle/PaddleDetection.git

# 安装其他依赖
%cd PaddleDetection
! pip install paddledet==2.0.1 -i https://mirror.baidu.com/pypi/simple
# 验证是否安装成功
! python ppdet/modeling/tests/test_architectures.py

Cloning into 'PaddleDetection'...
remote: Enumerating objects: 14575, done.
remote: Counting objects: 100% (14575/14575), done.
remote: Compressing objects: 100% (6264/6264), done.
remote: Total 14575 (delta 10731), reused 11457 (delta 8175), pack-reused 0
Receiving objects: 100% (14575/14575), 132.71 MiB | 17.54 MiB/s, done.
Resolving deltas: 100% (10731/10731), done.
Checking connectivity... done.
/home/aistudio/PaddleDetection
Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 317kB 19.1MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 15.8MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278372 sha256=7206726bde667c91cd54152b01d83f9265e07abcab1b2f444e8bfa59cd66c2c8
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl siz

In [7]:
%cd /home/aistudio/
! unzip -qo data/data85130/常规赛：PALM眼底彩照中黄斑中央凹定位.zip
! mv 常规赛：PALM眼底彩照中黄斑中央凹定位 dataset

! mkdir ~/PaddleDetection/dataset/palm
! mkdir ~/PaddleDetection/dataset/palm/annotations
! mkdir ~/PaddleDetection/dataset/palm/images
# images
! cp dataset/Train/fundus_image/* PaddleDetection/dataset/palm/images

/home/aistudio


In [10]:
# annotations
from xml.etree.ElementTree import Element
from xml.etree.ElementTree import SubElement
from xml.etree.ElementTree import ElementTree
from xml.dom import minidom
from PIL import Image

def geneXml(imgName, x, y, width, height):
    img_name = imgName.split('.')[0]
    # generate root node
    root = Element('annotation')
    # frame
    frame = SubElement(root, 'frame')
    frame.text = img_name
    # object
    objects = SubElement(root, 'object')
    # object-name
    name = SubElement(objects, 'name')
    name.text = "1"
    # object-bndbox
    bndbox = SubElement(objects, 'bndbox')
    xmin = SubElement(bndbox, 'xmin')
    xmin.text = str(int(x)-20)
    ymin = SubElement(bndbox, 'ymin')
    ymin.text = str(int(y)-20)
    xmax = SubElement(bndbox, 'xmax')
    xmax.text = str(int(x)+20)
    ymax = SubElement(bndbox, 'ymax')
    ymax.text = str(int(y)+20)
    # size
    size = SubElement(root, 'size')
    widths = SubElement(size, 'width')
    widths.text = str(width)
    heights = SubElement(size, 'height')
    heights.text = str(height)
    # write out xml data
    prettyXml(root, '\t', '\n')
    tree = ElementTree(root)
    tree.write(f'PaddleDetection/dataset/palm/annotations/{img_name}.xml', encoding='utf-8')

def prettyXml(element, indent='\t', newline='\n', level = 0): 
    # 判断element是否有子元素
    if element:
        # 如果element的text没有内容      
        if element.text == None or element.text.isspace():     
            element.text = newline + indent * (level + 1)      
        else:    
            element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * (level + 1)    
    # 此处两行如果把注释去掉，Element的text也会另起一行 
    #else:     
        #element.text = newline + indent * (level + 1) + element.text.strip() + newline + indent * level    
    temp = list(element) # 将elemnt转成list    
    for subelement in temp:    
        # 如果不是list的最后一个元素，说明下一个行是同级别元素的起始，缩进应一致
        if temp.index(subelement) < (len(temp) - 1):     
            subelement.tail = newline + indent * (level + 1)    
        else:  # 如果是list的最后一个元素， 说明下一行是母元素的结束，缩进应该少一个    
            subelement.tail = newline + indent * level   
        # 对子元素进行递归操作 
        prettyXml(subelement, indent, newline, level = level + 1) 

from tqdm import tqdm
import pandas as pd
data = pd.read_excel('dataset/Train/Fovea_Location_train.xlsx')
for i in tqdm(data.index):
    imgName, x, y = data.loc[i, ['imgName', 'Fovea_X', 'Fovea_Y']]
    width, height = Image.open(f'dataset/Train/fundus_image/{imgName}').size
    geneXml(imgName, x, y, width, height)

100%|██████████| 800/800 [00:00<00:00, 1116.89it/s]


In [11]:
import os, random
from tqdm import tqdm
def splitPic():
    file_list = os.listdir('PaddleDetection/dataset/palm/images/')
    random.shuffle(file_list)
    n = len(file_list)
    train_list = file_list[:int(n*0.8)]
    val_list = file_list[int(n*0.8):]
    print(n, len(train_list), len(val_list))
    train_txt = open('PaddleDetection/dataset/palm/train.txt', 'w')
    for file in tqdm(train_list, desc='Train'):
        file_name = file.split('.')[0]
        train_txt.write(f'./images/{file_name}.jpg ./annotations/{file_name}.xml\n')
    train_txt.close()
    val_txt = open('PaddleDetection/dataset/palm/valid.txt', 'w')
    for file in tqdm(val_list, desc='Val'):
        file_name = file.split('.')[0]
        val_txt.write(f'./images/{file_name}.jpg ./annotations/{file_name}.xml\n')
    val_txt.close()
splitPic()

Val: 100%|██████████| 160/160 [00:00<00:00, 630722.41it/s]


800 640 160


In [17]:
with open('/home/aistudio/PaddleDetection/dataset/palm/label_list.txt', 'w') as f:
    f.write('1')

# 训练数据并提交测试数据结果
- 修改对应配置
- 训练数据
- 预测测试数据
- 按照提交形式处理预测数据
- 提交结果

**模型构建思路及调优过程**
- 修改的配置主要是线程数（加快训练速度）和对应的数据集还有训练轮数（只设置了一个较低值50，因为训练时间有限）
- 在预测数据的时候调整了draw_threshold的值，这样就可以避免很多错误的预测
- 其实最后预测出来的数据很少，分数也很低，还有的思路就是把图片调白然后训练，因为现在的点实在是不好认出来

In [12]:
! cp PaddleDetection/configs/yolov3/yolov3_mobilenet_v1_roadsign.yml PaddleDetection/configs/yolov3/yolov3_palm_voc.yml
! cp PaddleDetection/configs/yolov3/_base_/yolov3_reader.yml PaddleDetection/configs/yolov3/_base_/yolov3_palm_reader.yml
! cp PaddleDetection/configs/yolov3/_base_/yolov3_mobilenet_v1.yml PaddleDetection/configs/yolov3/_base_/yolov3_palm_mobilenet.yml
! cp PaddleDetection/configs/datasets/roadsign_voc.yml PaddleDetection/configs/datasets/palm_voc.yml

**下面是修改的配置文件**
```
### yolov3_palm_voc.yml ###
_BASE_: [
  '../datasets/palm_voc.yml',
  '../runtime.yml',
  '_base_/yolov3_palm_mobilenet.yml',
  '_base_/yolov3_palm_reader.yml',
]
# pretrain_weights: https://paddledet.bj.bcebos.com/models/yolov3_mobilenet_v1_270e_coco.pdparams
# weights: output/yolov3_mobilenet_v1_roadsign/model_final
pretrain_weights:: ~/PaddleDetection/output/yolov3_palm_voc/model_final.pdparams

YOLOv3Loss:
  ignore_thresh: 0.7
  label_smooth: true

snapshot_epoch: 2
epoch: 50

LearningRate:
  base_lr: 0.001
  schedulers:
  - !PiecewiseDecay
    gamma: 0.1
    milestones: [32, 36]
  - !LinearWarmup
    start_factor: 0.3333333333333333
    steps: 100

OptimizerBuilder:
  optimizer:
    momentum: 0.9
    type: Momentum
  regularizer:
    factor: 0.0005
    type: L2


### yolov3_palm_reader.yml ###
worker_num: 3
TrainReader:
  inputs_def:
    num_max_boxes: 1
  sample_transforms:
    - Decode: {}
    - Mixup: {alpha: 1.5, beta: 1.5}
    - RandomDistort: {}
    - RandomExpand: {fill_value: [123.675, 116.28, 103.53]}
    - RandomCrop: {}
    - RandomFlip: {}
  batch_transforms:
    - BatchRandomResize: {target_size: [320, 352, 384, 416, 448, 480, 512, 544, 576, 608], random_size: True, random_interp: True, keep_ratio: False}
    - NormalizeBox: {}
    - PadBox: {num_max_boxes: 1}
    - BboxXYXY2XYWH: {}
    - NormalizeImage: {mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225], is_scale: True}
    - Permute: {}
    - Gt2YoloTarget: {anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]], anchors: [[10, 13], [16, 30], [33, 23], [30, 61], [62, 45], [59, 119], [116, 90], [156, 198], [373, 326]], downsample_ratios: [32, 16, 8]}
  batch_size: 24
  shuffle: true
  drop_last: true
  mixup_epoch: 250
  use_shared_memory: true

EvalReader:
  inputs_def:
    num_max_boxes: 1
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: [608, 608], keep_ratio: False, interp: 2}
    - NormalizeImage: {mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225], is_scale: True}
    - Permute: {}
  batch_size: 1
  drop_empty: false

TestReader:
  inputs_def:
    image_shape: [3, 608, 608]
  sample_transforms:
    - Decode: {}
    - Resize: {target_size: [608, 608], keep_ratio: False, interp: 2}
    - NormalizeImage: {mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225], is_scale: True}
    - Permute: {}
  batch_size: 1


### yolov3_palm_mobilenet.yml ####
architecture: YOLOv3
pretrain_weights: https://paddledet.bj.bcebos.com/models/pretrained/MobileNetV1_pretrained.pdparams
norm_type: sync_bn

YOLOv3:
  backbone: MobileNet
  neck: YOLOv3FPN
  yolo_head: YOLOv3Head
  post_process: BBoxPostProcess

MobileNet:
  scale: 1
  feature_maps: [4, 6, 13]
  with_extra_blocks: false
  extra_block_filters: []

# use default config
# YOLOv3FPN:

YOLOv3Head:
  anchors: [[10, 13], [16, 30], [33, 23],
            [30, 61], [62, 45], [59, 119],
            [116, 90], [156, 198], [373, 326]]
  anchor_masks: [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
  loss: YOLOv3Loss

YOLOv3Loss:
  ignore_thresh: 0.7
  downsample: [32, 16, 8]
  label_smooth: false

BBoxPostProcess:
  decode:
    name: YOLOBox
    conf_thresh: 0.005
    downsample_ratio: 32
    clip_bbox: true
  nms:
    name: MultiClassNMS
    keep_top_k: 100
    score_threshold: 0.01
    nms_threshold: 0.45
    nms_top_k: 1000



### palm_voc.yml ###
metric: VOC
map_type: integral
num_classes: 1

TrainDataset:
  !VOCDataSet
    dataset_dir: dataset/palm
    anno_path: train.txt
    label_list: label_list.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

EvalDataset:
  !VOCDataSet
    dataset_dir: dataset/palm
    anno_path: valid.txt
    label_list: label_list.txt
    data_fields: ['image', 'gt_bbox', 'gt_class', 'difficult']

TestDataset:
  !ImageFolder
    anno_path: dataset/palm/label_list.txt
```

In [18]:
%cd /home/aistudio/PaddleDetection/
! python tools/train.py -c configs/yolov3/yolov3_palm_voc.yml --eval --use_vdl=True --vdl_log_dir="./output"

/home/aistudio/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0603 10:09:15.575665  2502 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0603 10:09:15.580976  2502 device_context.cc:422] device: 0, cuDNN Version: 7.6.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ppdet/data/reader.py:190: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  logger.warn("Shared memory size is less than 1G, "
[06/03 10:09:18] reader WARNING: Shared memory size is less than 1G, disable 

In [19]:
%cd /home/aistudio/PaddleDetection/
! python tools/infer.py -c configs/yolov3/yolov3_palm_voc.yml -o weights=output/yolov3_palm_voc/best_model.pdparams --infer_dir=/home/aistudio/dataset/PALM-Testing400-Images --save_txt=True --output_dir=output/yolov3_palm_voc/pred/ --draw_threshold=0.5

/home/aistudio/PaddleDetection
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0603 14:00:14.117980 17677 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0603 14:00:14.123402 17677 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[06/03 14:00:18] ppdet.utils.checkpoint INFO: Finish loading model weights: output/yolov3_palm_voc/best_model.pdparams
[06/03 14:00:18] train INFO: Found 400 inference images in total.
[06/03 14:00:19] ppdet.engine INFO: Detection bbox results save in output/yolov3_palm_voc/pred/T0292.jpg
[06/03 1

In [20]:
from PIL import Image
import os
submit = open('/home/aistudio/dataset/submit.csv', 'w')
submit.write("FileName,Fovea_X,Fovea_Y\n")
for f in os.listdir('/home/aistudio/PaddleDetection/output/yolov3_palm_voc/pred/'):
    img_name = f.split('.')[0]
    if f.split('.')[1] == 'jpg':
        continue
    size = os.path.getsize(f'/home/aistudio/PaddleDetection/output/yolov3_palm_voc/pred/{f}')
    if size > 0:
        pred = open(f'/home/aistudio/PaddleDetection/output/yolov3_palm_voc/pred/{f}')
        line = pred.readline()
        _, _, xmin, ymin, xmax, ymax = line.strip().strip().split(' ')
        x = (float(xmin) + float(ymax)) / 2
        y = (float(ymin) + float(ymax)) / 2
        pred.close()
    else:
        img = Image.open(f'/home/aistudio/PaddleDetection/output/yolov3_palm_voc/pred/{img_name}.jpg')
        width, height = img.size
        x = width / 2
        y = height / 2
    submit.write(f"{img_name}.jpg,{x},{y}\n")
submit.close()